# Rest API using the Pythorch classifier model and tfidf vectorizer

In [17]:
from flask import Flask
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
from flask import request

In [7]:
# Declare class

input_size = 466
output_size = 2
hidden_size = 500

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)
        
            
    def forward(self, X):
        X = torch.relu(self.fc1(X))
        X = torch.relu((self.fc2(X)))
        X = self.fc3(X)
        
        return F.log_softmax(X, dim=1)

In [9]:
# Declare a class for the Pytorch model
model = Net()

In [12]:
# Load pytorch dictonary
model.load_state_dict(torch.load('text_classifier_pytorch'))

<All keys matched successfully>

In [13]:
app = Flask(__name__)

In [14]:
# Load tfidf model

with open('tfidfmodel.pickle', 'rb') as file:
    tfidf = pickle.load(file)

In [15]:
# Method for read the request

@app.route('/predict', methods=['POST'])
def predict_sentence():
    request_data = request.get_json(force = True)
    text = request_data['sentence']
    print("Sentence is:", text)
    text_list = []
    text_list.append(text)
    print(text_list)
    numeric_text = tfidf.transform(text_list).toarray()
    output = model(torch.from_numpy(numeric_text).float())  # Pytorch predict the output
    print("Printing predictions...")
    print(output[:, 0][0])
    print(output[:, 1][0])
    sentiment = "unknown"
    if torch.gt(output[:, 0][0], output[:, 1][0]):
        print("Negative prediction")
        sentiment = "Negative"
    else:
        print("Positive")
        sentiment = "Positive prediction"
    print("Printing prediction ...")
    print(sentiment)
    return "The prediction is {}".format(sentiment)
    
    

In [18]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2021 16:30:33] "POST /predict HTTP/1.1" 200 -


Sentence is: good day in poland
['good day in poland']
Printing predictions...
tensor(-24.2465, grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
Positive
Printing prediction ...
Positive prediction


127.0.0.1 - - [12/Jun/2021 16:30:41] "POST /predict HTTP/1.1" 200 -


Sentence is: bad day in poland
['bad day in poland']
Printing predictions...
tensor(0., grad_fn=<SelectBackward>)
tensor(-21.5121, grad_fn=<SelectBackward>)
Negative prediction
Printing prediction ...
Negative
